In [1]:
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import numpy as np
from PIL import Image, ImageTk
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import os
from collections import OrderedDict

class ArtisticColorizationSystem:
    def __init__(self):
        # Available artistic styles
        self.styles = {
            "Van Gogh (Starry Night)": "vangogh",
            "Monet (Water Lilies)": "monet",
            "Picasso (Cubist)": "picasso",
            "Hokusai (The Great Wave)": "hokusai",
            "Rembrandt (Chiaroscuro)": "rembrandt",
            "Modern Pop Art": "popart",
            "Impressionist": "impressionist",
            "No Style (Natural Color)": "natural"
        }

        # Initialize models
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.colorization_model = None
        self.style_transfer_model = None
        self.vgg19 = None

        # GUI components
        self.root = None
        self.input_image = None
        self.output_image = None
        self.style_var = None

        self.setup_models()

    def setup_models(self):
        """Initialize the colorization and style transfer models"""
        print("Setting up models...")

        # Setup VGG19 for style extraction (simplified)
        self.vgg19 = models.vgg19(pretrained=True).features.to(self.device).eval()

        # For the demo, we'll use a simplified approach
        # In production, you would load pretrained colorization and style transfer models
        print("Models initialized (simulated for demo)")

CORE MODELS:

In [2]:
class ColorizationNet(nn.Module):
    """U-Net based architecture for colorization"""
    def __init__(self, input_channels=1, output_channels=2):
        super(ColorizationNet, self).__init__()

        # Encoder
        self.encoder1 = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )

        # Decoder
        self.decoder2 = nn.Sequential(
            nn.Conv2d(384, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )

        self.decoder1 = nn.Sequential(
            nn.Conv2d(192, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, output_channels, kernel_size=1)
        )

    def forward(self, x):
        # Encoder path
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(enc1)

        # Bottleneck
        bottleneck = self.bottleneck(enc2)

        # Decoder path with skip connections
        dec2 = self.decoder2(torch.cat([bottleneck, enc2], dim=1))
        dec1 = self.decoder1(torch.cat([dec2, enc1], dim=1))

        return dec1

In [4]:
class StyleTransfer:
    """Apply artistic styles to colorized images"""
    @staticmethod
    def extract_features(image, model):
        """Extract features from VGG19 layers"""
        features = []
        x = image
        for i, layer in enumerate(model):
            x = layer(x)
            if i in {1, 6, 11, 20, 29}:  # Selected layers for style/content
                features.append(x)
        return features

    @staticmethod
    def gram_matrix(tensor):
        """Compute Gram matrix for style representation"""
        _, d, h, w = tensor.size()
        tensor = tensor.view(d, h * w)
        gram = torch.mm(tensor, tensor.t())
        return gram

    @staticmethod
    def apply_style(colorized_img, style_name):
        """Apply specific artistic style to image"""
        # Convert to LAB and then to RGB for processing
        if isinstance(colorized_img, np.ndarray):
            colorized_img = torch.from_numpy(colorized_img).float().to('cpu')

        # In production, this would use actual style transfer models
        # For demo, we apply simulated style effects
        img_np = colorized_img.numpy().transpose(1, 2, 0)

        # Apply different filters based on selected style
        if style_name == "vangogh":
            # Simulate Van Gogh's brush strokes
            return StyleTransfer.apply_vangogh_effect(img_np)
        elif style_name == "monet":
            # Simulate Monet's impressionism
            return StyleTransfer.apply_monet_effect(img_np)
        elif style_name == "popart":
            # Simulate pop art effect
            return StyleTransfer.apply_popart_effect(img_np)
        elif style_name == "natural":
            # No style, return original
            return img_np
        else:
            # Default mild artistic effect
            return StyleTransfer.apply_default_artistic(img_np)

    @staticmethod
    def apply_vangogh_effect(img):
        """Simulate Van Gogh's style"""
        # This would be replaced with actual neural style transfer
        # For demo, using OpenCV filters
        result = cv2.stylization(img, sigma_s=60, sigma_r=0.6)
        result = cv2.detailEnhance(result, sigma_s=10, sigma_r=0.15)
        return result

    @staticmethod
    def apply_monet_effect(img):
        """Simulate Monet's impressionist style"""
        result = cv2.edgePreservingFilter(img, flags=1, sigma_s=60, sigma_r=0.4)
        # Add warm impressionist tone
        result = cv2.applyColorMap(result, cv2.COLORMAP_SUMMER)
        return result

    @staticmethod
    def apply_popart_effect(img):
        """Apply pop art style"""
        # Posterization effect
        result = cv2.medianBlur(img, 3)
        Z = result.reshape((-1, 3))
        Z = np.float32(Z)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        K = 8
        _, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        center = np.uint8(center)
        result = center[label.flatten()].reshape((result.shape))
        return result

    @staticmethod
    def apply_default_artistic(img):
        """Default artistic enhancement"""
        result = cv2.detailEnhance(img, sigma_s=10, sigma_r=0.15)
        return result

In [5]:
def main():
    """Main application entry point"""
    print("Starting Artistic Style Transfer in Colorization System...")
    print("=" * 60)

    # Check for required packages
    try:
        import tkinter
        import torch
        import torchvision
        import PIL
        import cv2
        import numpy as np
    except ImportError as e:
        print(f"Error: Missing required package - {e}")
        print("Please install required packages:")
        print("pip install torch torchvision pillow opencv-python numpy")
        return

    # Create and run the system
    system = ArtisticColorizationSystem()
    gui = ArtisticColorizationGUI(system)
    gui.run()

if __name__ == "__main__":
    main()

Starting Artistic Style Transfer in Colorization System...
Setting up models...


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


 74%|███████▍  | 407M/548M [00:02<00:00, 200MB/s]


KeyboardInterrupt: 

GUI Interface


In [7]:
class ArtisticColorizationGUI:
    """Graphical User Interface for the system"""
    def __init__(self, system):
        self.system = system
        self.root = tk.Tk()
        self.root.title("Artistic Style Transfer in Colorization")
        self.root.geometry("1200x800")

        # Configure style
        self.setup_styles()

        # Variables
        self.input_image_path = None
        self.output_image = None
        self.style_var = tk.StringVar(value="No Style (Natural Color)")

        # Create GUI
        self.create_widgets()

    def setup_styles(self):
        """Setup ttk styles"""
        style = ttk.Style()
        style.theme_use('clam')

        # Custom colors
        bg_color = "#2c3e50"
        fg_color = "#ecf0f1"
        accent_color = "#3498db"

        style.configure('Title.TLabel',
                       font=('Helvetica', 16, 'bold'),
                       background=bg_color,
                       foreground=fg_color)

        style.configure('Subtitle.TLabel',
                       font=('Helvetica', 12),
                       background=bg_color,
                       foreground=accent_color)

    def create_widgets(self):
        """Create all GUI widgets"""
        # Main container
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Title
        title_label = ttk.Label(main_frame,
                               text="Artistic Style Transfer in Colorization",
                               style='Title.TLabel')
        title_label.pack(pady=(0, 20))

        # Content frame
        content_frame = ttk.Frame(main_frame)
        content_frame.pack(fill=tk.BOTH, expand=True)

        # Left panel - Input
        left_panel = ttk.LabelFrame(content_frame, text="Input Image", padding=10)
        left_panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)

        # Input image display
        self.input_canvas = tk.Canvas(left_panel, width=400, height=400,
                                     bg='#34495e', relief=tk.SUNKEN)
        self.input_canvas.pack(pady=10)
        self.input_canvas.create_text(200, 200,
                                     text="No Image Selected",
                                     fill="white",
                                     font=('Helvetica', 12))

        # Upload button
        upload_btn = ttk.Button(left_panel, text="Upload Grayscale Image",
                               command=self.upload_image)
        upload_btn.pack(pady=10)

        # Style selection
        style_frame = ttk.LabelFrame(content_frame, text="Artistic Style Selection", padding=10)
        style_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)

        # Style dropdown
        style_label = ttk.Label(style_frame, text="Choose Artistic Style:")
        style_label.pack(anchor=tk.W, pady=(0, 5))

        self.style_combo = ttk.Combobox(style_frame,
                                       textvariable=self.style_var,
                                       values=list(self.system.styles.keys()),
                                       state="readonly")
        self.style_combo.pack(fill=tk.X, pady=(0, 20))

        # Style preview images (simulated)
        preview_label = ttk.Label(style_frame, text="Style Previews:")
        preview_label.pack(anchor=tk.W, pady=(10, 5))

        preview_frame = ttk.Frame(style_frame)
        preview_frame.pack(fill=tk.BOTH, expand=True)

        # Right panel - Output
        right_panel = ttk.LabelFrame(content_frame, text="Colorized & Styled Output", padding=10)
        right_panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)

        # Output image display
        self.output_canvas = tk.Canvas(right_panel, width=400, height=400,
                                      bg='#34495e', relief=tk.SUNKEN)
        self.output_canvas.pack(pady=10)
        self.output_canvas.create_text(200, 200,
                                      text="Processed Image will appear here",
                                      fill="white",
                                      font=('Helvetica', 12))

        # Process button
        process_btn = ttk.Button(right_panel, text="Colorize & Apply Style",
                                command=self.process_image,
                                style='Accent.TButton')
        process_btn.pack(pady=20)

        # Progress bar
        self.progress = ttk.Progressbar(right_panel, mode='indeterminate')
        self.progress.pack(fill=tk.X, pady=10)

        # Status bar
        self.status_var = tk.StringVar(value="Ready to process images")
        status_bar = ttk.Label(main_frame, textvariable=self.status_var,
                              relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)

        # Control buttons frame
        control_frame = ttk.Frame(main_frame)
        control_frame.pack(fill=tk.X, pady=10)

        ttk.Button(control_frame, text="Save Output",
                  command=self.save_output).pack(side=tk.LEFT, padx=5)
        ttk.Button(control_frame, text="Reset",
                  command=self.reset).pack(side=tk.LEFT, padx=5)
        ttk.Button(control_frame, text="Exit",
                  command=self.root.quit).pack(side=tk.RIGHT, padx=5)

    def upload_image(self):
        """Handle image upload"""
        file_path = filedialog.askopenfilename(
            title="Select Grayscale Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff")]
        )

        if file_path:
            try:
                self.input_image_path = file_path
                img = Image.open(file_path)

                # Convert to grayscale if needed
                if img.mode != 'L':
                    img = img.convert('L')

                # Resize for display
                img.thumbnail((380, 380))
                self.input_display_img = ImageTk.PhotoImage(img)

                # Update canvas
                self.input_canvas.delete("all")
                self.input_canvas.create_image(200, 200,
                                             image=self.input_display_img)

                self.status_var.set(f"Loaded: {os.path.basename(file_path)}")

            except Exception as e:
                messagebox.showerror("Error", f"Failed to load image: {str(e)}")

    def process_image(self):
        """Process image with colorization and style transfer"""
        if not self.input_image_path:
            messagebox.showwarning("No Image", "Please upload an image first!")
            return

        # Start progress bar
        self.progress.start()
        self.status_var.set("Processing image...")
        self.root.update()

        try:
            # Load and preprocess image
            img = Image.open(self.input_image_path).convert('L')
            img_array = np.array(img)

            # Simulate colorization (in production, use actual model)
            colorized = self.simulate_colorization(img_array)

            # Get selected style
            style_name = self.system.styles[self.style_var.get()]

            # Apply style transfer
            styled = StyleTransfer.apply_style(colorized, style_name)

            # Convert to PIL Image for display
            if styled.shape[0] == 3:  # CHW format
                styled = styled.transpose(1, 2, 0)

            output_img = Image.fromarray((styled * 255).astype(np.uint8))
            output_img.thumbnail((380, 380))
            self.output_display_img = ImageTk.PhotoImage(output_img)

            # Update canvas
            self.output_canvas.delete("all")
            self.output_canvas.create_image(200, 200,
                                          image=self.output_display_img)

            self.status_var.set(f"Applied {self.style_var.get()} style successfully!")
            self.output_image = styled

        except Exception as e:
            messagebox.showerror("Processing Error", f"Failed to process image: {str(e)}")
            self.status_var.set("Processing failed")

        finally:
            self.progress.stop()

    def simulate_colorization(self, grayscale_img):
        """Simulate colorization process (demo purposes)"""
        # In production, this would use the actual ColorizationNet
        # For demo, create a color version from grayscale
        if len(grayscale_img.shape) == 2:
            # Convert to 3-channel with color tint
            colorized = np.stack([grayscale_img] * 3, axis=-1)

            # Apply color tint based on image content
            mean_val = np.mean(grayscale_img)
            if mean_val < 85:
                # Dark image - warm tones
                colorized[:, :, 0] *= 0.9  # Reduce blue
                colorized[:, :, 1] *= 1.1  # Enhance green
            elif mean_val > 170:
                # Bright image - cool tones
                colorized[:, :, 0] *= 1.2  # Enhance blue
            else:
                # Mid tones - balanced
                colorized[:, :, 0] *= 1.1  # Slight blue tint
                colorized[:, :, 2] *= 0.9  # Reduce red

            # Normalize
            colorized = np.clip(colorized, 0, 255).astype(np.uint8)
            colorized = colorized.transpose(2, 0, 1) / 255.0

            return colorized
        return grayscale_img

    def save_output(self):
        """Save processed image"""
        if self.output_image is None:
            messagebox.showwarning("No Output", "No processed image to save!")
            return

        file_path = filedialog.asksaveasfilename(
            defaultextension=".png",
            filetypes=[("PNG files", "*.png"),
                      ("JPEG files", "*.jpg"),
                      ("All files", "*.*")]
        )

        if file_path:
            try:
                # Convert to proper format and save
                if isinstance(self.output_image, np.ndarray):
                    if self.output_image.shape[0] == 3:
                        img_to_save = (self.output_image.transpose(1, 2, 0) * 255).astype(np.uint8)
                    else:
                        img_to_save = (self.output_image * 255).astype(np.uint8)

                    Image.fromarray(img_to_save).save(file_path)
                    self.status_var.set(f"Image saved to {os.path.basename(file_path)}")

            except Exception as e:
                messagebox.showerror("Save Error", f"Failed to save image: {str(e)}")

    def reset(self):
        """Reset the application"""
        self.input_image_path = None
        self.output_image = None
        self.input_canvas.delete("all")
        self.output_canvas.delete("all")
        self.input_canvas.create_text(200, 200,
                                     text="No Image Selected",
                                     fill="white")
        self.output_canvas.create_text(200, 200,
                                      text="Processed Image will appear here",
                                      fill="white")
        self.style_var.set("No Style (Natural Color)")
        self.status_var.set("Ready to process images")

    def run(self):
        """Start the GUI application"""
        self.root.mainloop()

The `TclError` means that a graphical user interface (GUI) application is trying to run in a headless environment (like Colab) without a display server. To fix this, we will modify the `main` function to not initialize the `tkinter` GUI directly. If you need a graphical output, consider alternative visualization methods like displaying images directly in the notebook output using `matplotlib` or saving them to files.

In [ ]:
!pip install gradio torch torchvision Pillow opencv-python numpy


In [8]:

import gradio as gr
import cv2
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

def apply_style_effect(image, style_name):
    """Apply artistic style effects using OpenCV"""
    img_array = np.array(image)

    if style_name == "Van Gogh":
        # Stylization effect
        result = cv2.stylization(img_array, sigma_s=60, sigma_r=0.6)
    elif style_name == "Monet":
        # Impressionist effect
        result = cv2.edgePreservingFilter(img_array, flags=1, sigma_s=60, sigma_r=0.4)
    elif style_name == "Picasso":
        # Cubist-like effect
        result = cv2.detailEnhance(img_array, sigma_s=10, sigma_r=0.15)
    elif style_name == "Hokusai":
        # Woodblock print effect
        result = cv2.applyColorMap(img_array, cv2.COLORMAP_SUMMER)
    elif style_name == "Pop Art":
        # Posterization effect
        Z = img_array.reshape((-1,3))
        Z = np.float32(Z)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
        K = 8
        _, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        center = np.uint8(center)
        result = center[label.flatten()].reshape((img_array.shape))
    else:  # Natural
        result = img_array

    return Image.fromarray(result)

def colorize_and_style(input_image, style_choice):
    """Main processing function"""
    # Convert to grayscale first (simulating colorization input)
    if input_image.mode != 'L':
        grayscale = input_image.convert('L')
        # Convert back to RGB for demo (simulating colorization)
        colorized = Image.merge('RGB', [grayscale, grayscale, grayscale])
    else:
        colorized = Image.merge('RGB', [input_image, input_image, input_image])

    # Apply selected style
    styled_image = apply_style_effect(colorized, style_choice)

    return styled_image

# Create Gradio interface
iface = gr.Interface(
    fn=colorize_and_style,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Dropdown(
            choices=["Van Gogh", "Monet", "Picasso", "Hokusai", "Pop Art", "Natural"],
            value="Natural",
            label="Select Artistic Style"
        )
    ],
    outputs=gr.Image(type="pil", label="Styled Output"),
    title="🎨 Artistic Style Transfer in Colorization",
    description="Upload an image and select an artistic style to apply. The system will colorize (simulated) and apply the selected style.",
    examples=[
        ["https://images.unsplash.com/photo-1506744038136-46273834b3fb", "Van Gogh"],
        ["https://images.unsplash.com/photo-1519681393784-d120267933ba", "Monet"]
    ]
)

# Launch in Colab
iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://db5eb5de61e2f6be76.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://db5eb5de61e2f6be76.gradio.live
